In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [2]:
def get_perspective_transform(img, src, dst, size):
    """ 
    #---------------------
    # This function takes in an image with source and destination image points,
    # generates the transform matrix and inverst transformation matrix, 
    # warps the image based on that matrix and returns the warped image with new perspective, 
    # along with both the regular and inverse transform matrices.
    #
    """

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warp_img = cv2.warpPerspective(img, M, size, flags=cv2.INTER_LINEAR)

    return warp_img, M, Minv

In [3]:
def  abs_sobel_thresh(img_channel, orient='x', thresh=(20, 100), sobel_kernel=3):
    """
    Find edges that are aligned vertically and horizontally on the image

    :param img_channel: Channel from an image
    :param orient: Across which axis of the image are we detecting edges?
    :sobel_kernel: No. of rows and columns of the kernel (i.e. 3x3 small matrix)
    :return: Image with Sobel edge detection applied
    """
    # cv2.Sobel(input image, data type, prder of the derivative x, order of the
    # derivative y, small matrix used to calculate the derivative)
    if orient == 'x':
    # Will detect differences in pixel intensities going from 
        # left to right on the image (i.e. edges that are vertically aligned)
        sobel =  np.absolute(cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, sobel_kernel))
    if orient == 'y':
        # Will detect differences in pixel intensities going from 
        # top to bottom on the image (i.e. edges that are horizontally aligned)
        sobel =  np.absolute(cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, sobel_kernel))

    # Scale to 8-bit (0 - 255) then convert to type = np.uint8    
    scaled_sobel = np.uint8(255*sobel/np.max(sobel))
   
    # Create a binary mask where mag thresholds are met  
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 255

    # Return the result
    return binary_output

In [4]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    """
    #---------------------
    # This function takes in an image and optional Sobel kernel size, 
    # as well as thresholds for gradient magnitude. And computes the gradient magnitude, 
    # applies a threshold, and creates a binary output image showing where thresholds were met.
    #
    """
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)

    # Create a binary mask where mag thresholds are met    
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 255

    # Return the binary image
    return binary_output

In [5]:
def dir_thresh(img, sobel_kernel=3, thresh=(0.7, 1.3)):
    """
    #---------------------
    # This function applies Sobel x and y, 
    # then computes the direction of the gradient,
    # and then applies a threshold.
    #
    """
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Take the absolute value of the x and y gradients 
    # and calculate the direction of the gradient
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
   
    # Create a binary mask where direction thresholds are met 
    binary_output = np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 255
    
    # Return the binary image
    return binary_output.astype(np.uint8)

In [6]:
def get_combined_gradients(img, thresh_x, thresh_y, thresh_mag, thresh_dir):
    """
    #---------------------
    # This function isolates lane line pixels, by focusing on pixels
    # that are likely to be part of lane lines.
    # I am using Red Channel, since it detects white pixels very well. 
    #
    """
    rows, cols = img.shape[:2]
    
    # save cropped image for documentation
    temp = np.copy(img)
    temp = temp[:, 0:cols, 2]
    cv2.imwrite("./output_images/02_cropped.png", temp)

    R_channel = img[:, 0:cols, 2]   # focusing only on regions where lane lines are likely present

    sobelx = abs_sobel_thresh(R_channel, 'x', thresh_x)
    sobely = abs_sobel_thresh(R_channel, 'y', thresh_y)
    mag_binary = mag_thresh(R_channel, 3, thresh_mag)
    dir_binary = dir_thresh(R_channel, 15, thresh_dir)
    
    # debug
    #cv2.imshow('sobelx', sobelx)

    # combine sobelx, sobely, magnitude & direction measurements
    gradient_combined = np.zeros_like(dir_binary).astype(np.uint8)
    gradient_combined[((sobelx > 1) & (mag_binary > 1) & (dir_binary > 1)) | ((sobelx > 1) & (sobely > 1))] = 255  # | (R > 1)] = 255

    return gradient_combined

In [7]:
def channel_thresh(channel, thresh=(80, 255)):
    """
    #---------------------
    # This function takes in a channel of an image and
    # returns thresholded binary image
    # 
    """
    binary = np.zeros_like(channel)
    binary[(channel > thresh[0]) & (channel <= thresh[1])] = 255
    return binary

In [8]:
def get_combined_hls(img, th_h, th_l, th_s):
    """
    #---------------------
    # This function takes in an image, converts it to HLS colorspace, 
    # extracts individual channels, applies thresholding on them
    #
    """

    # convert to hls color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)

    rows, cols = img.shape[:2]
    
    # trying to use Red channel info to improve results
    #R = img[220:rows - 12, 0:cols, 2]
    #_, R = cv2.threshold(R, 180, 255, cv2.THRESH_BINARY)
    
    H = hls[:, 0:cols, 0]
    L = hls[:, 0:cols, 1]
    S = hls[:, 0:cols, 2]

    h_channel = channel_thresh(H, th_h)
    l_channel = channel_thresh(L, th_l)
    s_channel = channel_thresh(S, th_s)
    
    # debug
    #cv2.imshow('Thresholded S channel', s_channel)

    # Trying to use Red channel, it works even better than S channel sometimes, 
    # but in cases where there is shadow on road and road color is different, 
    # S channel works better. 
    hls_comb = np.zeros_like(s_channel).astype(np.uint8)
    hls_comb[((s_channel > 1) & (l_channel == 0)) | ((s_channel == 0) & (h_channel > 1) & (l_channel > 1))] = 255 
    # trying to use both S channel and R channel
    #hls_comb[((s_channel > 1) & (h_channel > 1)) | (R > 1)] = 255
   
    # return combined hls image 
    return hls_comb


In [9]:
def combine_grad_hls(grad, hls):
    """ 
    #---------------------
    # This function combines gradient and hls images into one.
    # For binary gradient image, if pixel is bright, set that pixel value in reulting image to 255
    # For binary hls image, if pixel is bright, set that pixel value in resulting image to 255 
    # Edit: Assign different values to distinguish them
    # 
    """
    result = np.zeros_like(hls).astype(np.uint8)
    result[(grad > 1)] = 100
    result[(hls > 1)] = 255

    return result

In [11]:
def region_of_interest(img, vertices):

    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


In [10]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    # Get the size of the figure
    xsize, ysize = img.shape[1], img.shape[0]
    
    # Fit two linear function for left/right lanes
    x_left = []
    y_left = []
    x_right = []
    y_right = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if abs(x1-x2) == 0 or abs((y1-y2)/(x1-x2)) < 0.5:
                # Skip vertical line and the dot 
                break
                
            elif (y2-y1)/(x2-x1) > 0:
                # Right lane
                x_right.append(x2)
                x_right.append(x1)
                y_right.append(y2)
                y_right.append(y1)
            else:
                # Left lane
                x_left.append(x2)
                x_left.append(x1)
                y_left.append(y2)
                y_left.append(y1)
                
    y_start = ysize//2 + 120
    y_end = ysize
    if x_left:
        k_left, b_left = np.polyfit(x_left, y_left, 1)
        x_start_left = int((y_start-b_left)/k_left)
        x_end_left = int((y_end-b_left)/k_left)
        cv2.line(img, (x_start_left, y_start), (x_end_left, ysize), color, thickness)
    if x_right:
        k_right, b_right = np.polyfit(x_right, y_right, 1)
        x_start_right = int((y_start-b_right)/k_right)
        x_end_right = int((y_end-b_right)/k_right)
        cv2.line(img, (x_start_right, y_start), (x_end_right, ysize), color, thickness)
    
    
    try:
        center_lane = (x_end_right + x_end_left) / 2
        lane_width = x_end_right - x_end_left


        center_car = xsize / 2
        if center_lane > center_car:
            deviation = 'Vehicle is '+ str(round(abs(center_lane - center_car)*3.7/lane_width, 3)) + 'm Left of center'
        elif center_lane < center_car:
            deviation = 'Vehicle is '+ str(round(abs(center_lane - center_car)*3.7/lane_width, 3)) + 'm Right of center'
        else:
            deviation = 'by 0 (Centered)'

        cv2.putText(img, deviation, (10, 63), cv2.FONT_HERSHEY_SIMPLEX, 0.50, (100, 100, 100), 1)
    except:pass
    
    try:
        contours = np.array([[x_end_left,ysize], [x_start_left,y_start], [x_start_right,y_start], [x_end_right,ysize]])
        cv2.fillPoly(img, pts = [contours], color =(0,200,0))
    except:pass
    

In [12]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

In [13]:
def weighted_img(img, initial_img, α=0.5, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [14]:
def draw_lane_lines(image):
    
    xsize, ysize = image.shape[1], image.shape[0]
    
    th_sobelx, th_sobely, th_mag, th_dir = (35, 100), (30, 255), (30, 255), (0.7, 1.3)
    th_h, th_l, th_s = (10, 100), (0, 60), (85, 255)
    
    combined_gradient = get_combined_gradients(image, th_sobelx, th_sobely, th_mag, th_dir)

    combined_hls = get_combined_hls(image, th_h, th_l, th_s)

    combined_result = combine_grad_hls(combined_gradient, combined_hls)

    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(combined_result, low_threshold, high_threshold)
    
    vertices = np.array([[(xsize//2-90,ysize//2+120),(100,ysize),(xsize-60, ysize),(xsize//2+130,ysize//2+120)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 30 #minimum number of pixels making up a line
    max_line_gap = 20   # maximum gap in pixels between connectable line segments
    line_img = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
    
    img_with_lines = weighted_img(line_img, image, α=0.8, β=1., γ=0.)
    
    return img_with_lines

In [59]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [60]:
def process_image(image):
    
    result = draw_lane_lines(image)

    return result

In [61]:
white_output = 'project_video_out2.mp4'

clip1 = VideoFileClip("project_video.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)

t:   0%|                                                                    | 2/1260 [00:00<01:45, 11.94it/s, now=None]

Moviepy - Building video project_video_out2.mp4.
Moviepy - Writing video project_video_out2.mp4



Moviepy - Done !
Moviepy - video ready project_video_out2.mp4
Wall time: 3min 57s
